---
# Intertrial Variability: 2. Data Analysis

In [ ]:
import sys; sys.path.insert(1, '../')

from functools import partial
import numpy as np
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(font_scale=1.,style='ticks',context='notebook',font= 'georgia')
from scipy.stats import ttest_ind
import pingouin
from scripts.preproc import *
from scripts.viz import *
from sklearn import linear_model
from scipy.signal import detrend

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Load data

In [ ]:
fn = '../processed/data_preprocessed.pkl'

with open(fn, 'rb') as f:
    [all_data_raw, all_data_epoch] = pkl.load(f)
# Load AQ and EQ data
fn = 'D:/data/Psychiatrie_Autismus_2012/AQ_EQ.json'

AQ, EQ = get_AQ_EQ(fn, all_data_epoch)

# Convert Dict to list cause thats how i like it
all_data_epoch = [list(all_data_epoch['control'].values()), list(all_data_epoch['asd'].values())]
info = all_data_epoch[0][0].info
times = all_data_epoch[0][0].times
ch_names = all_data_epoch[0][0].ch_names

# ITV

## Compute

In [ ]:
ch_picks = ['O1', 'Oz', 'O2']
ch_indices = [ch_names.index(ch) for ch in ch_picks]
conds = [['SF', 'LF'], ['GPL', 'GPS']]
cond_labels = ['Frequent checkers',  'Grey blanks']

time_range = [-0.2, 0.5]
pnt_range = np.arange(*[np.argmin(np.abs(times-t)) for t in time_range])
scaler = 1e6

itv_conds = [[np.stack(list(map(partial(calc_itv, cond=cond, norm_type=None, relative=False), epochs)), axis=0) 
            for epochs in all_data_epoch] 
            for cond in conds]


## Plot

In [ ]:
figsize = (15, 5)
fig = plt.figure(num='ITV', figsize=figsize)
sns.set(font_scale=1.4)
for i, (cond, itv) in enumerate(zip(conds, itv_conds)):
    itv = [scaler * x[:, :, pnt_range] for x in itv]
    N = 121+i
    ax = plt.subplot(N)
    plot_itv(itv, times[pnt_range], ax, info, cond=cond, title=cond_labels[i], alpha=0.05, ch_picks=ch_picks)
    ax.set_ylim((0, plt.ylim()[1]))
    if i == 0:
        ax.set_xlabel('Time [s]')
        ax.set_ylabel('ITV [\u03BCV]')
    
ax.legend(bbox_to_anchor=(1,1), loc="upper left")
plt.tight_layout()

# multipage(r'C:\Users\Lukas\Documents\VariabilityAnalysis\figures\paper\for_review\ITV.pdf', figs=[fig], dpi=300, png=True)

## Prepare Data Frame

In [ ]:
ch_picks = ['O1', 'Oz', 'O2']
ch_indices = [ch_names.index(ch) for ch in ch_picks]
conds = [['SF', 'LF'], ['GPL', 'GPS']]
cond_labels = ['Frequent checkers',  'Grey blanks']


# Subject List
subjects = np.concatenate([
    ['Control_'+str(i+1) for i in range(len(all_data_epoch[0]))], 
    ['ASD_'+str(i+1) for i in range(len(all_data_epoch[1]))]
    ])
groups = np.concatenate([
    ['Control' for _ in range(len(all_data_epoch[0]))], 
    ['ASD' for _ in range(len(all_data_epoch[1]))]
    ])

df_list = []
# loop through Stimuli
for i, (itv, cond, cond_label) in enumerate(zip(itv_conds, conds, cond_labels)):
    # Prepare Data of the current stimulus    
    data = np.concatenate([np.mean(itv[0][:, ch_indices, :], axis=-1), np.mean(itv[1][:, ch_indices, :], axis=-1)], axis=0)
    
    df = pd.DataFrame(data, columns=ch_picks)
    df['Subject'] = subjects
    df['Group'] = groups
    df['Stimulus'] = [cond_label] * data.shape[0]
    # Melt individual electrode columns into a single Column
    df_itv = df.melt(value_vars=ch_picks, 
                    id_vars=['Subject', 'Group', 'Stimulus'], 
                    value_name='ITV', var_name='Electrode')
    df_list.append(df_itv)
# Concatenate DataFrames of the different stimuli
df_itv = pd.concat(df_list)

## Statistics

In [ ]:
dependent_variable = 'ITV'
factors = ['Group', 'Stimulus']
aov = df_itv.anova(dv=dependent_variable, between=factors, detailed=True, effsize='n2')


# post-hoc test
post_hoc = df_itv.pairwise_tukey(dv=dependent_variable, between='Group')
display(aov)
print('\nPost-Hoc:\n')
display(post_hoc)

# ETV

## Compute

In [ ]:
ch_picks = ['O1', 'Oz', 'O2']
ch_indices = [ch_names.index(ch) for ch in ch_picks]
conds = ['SF', 'LF', 'GPL', 'GPS']
cond_labels = ['Small frequent', 'Large frequent', 'Grey screen L', 'Grey screen S']

time_range = [-0.2, 0.5]
pnt_range = [np.argmin(np.abs(times-time_range[0])), 
            np.argmin(np.abs(times-time_range[1]))]
pnt_range = np.arange(*pnt_range)

# Calc ETV
etv_conds = [[list(map(partial(calc_etv, cond=cond), epochs)) 
        for epochs in all_data_epoch] 
        for cond in conds]


## Plot

In [ ]:
ylim = (4, 8.5)
scaler = 1e6

figsize = (12, 7)
fig = plt.figure(num='ETV', figsize=figsize)
sns.set(font_scale=1.2)
plt.suptitle('Evolving Inter-Trial Variability')
for i, (etv, cond) in enumerate(zip(etv_conds, conds)):
    params = dict(cond=cond)
    
    # Cut of trials so all participants have the same amount
    min_tr = min([min([sub.shape[0] for sub in eee]) for eee in etv])
    etv = [np.stack([sub[:min_tr, ch_indices, :]*scaler for sub in eee], axis=0) for eee in etv]
    # Average over time and re-arrange
    etv = [np.swapaxes(np.mean(group[..., pnt_range], axis=-1), 1, 2) for group in etv]
    # Plot
    N = 221+i
    ax = plt.subplot(N)
    plot_etv(etv, ax, info, cond=cond, title=cond_labels[i])
    ax.set_ylim(ylim)
    if i == 2:
        ax.set_xlabel('Trial No.')
        ax.set_ylabel('ETV [\u03BCV]')
    
ax.legend(bbox_to_anchor=(1,1), loc="upper left")
plt.tight_layout(pad=2)

# multipage(r'C:\Users\Lukas\Documents\VariabilityAnalysis\figures\paper\for_review\ETV.pdf', figs=[fig], dpi=300, png=True)

## Prepare Data Frame

In [ ]:
conds = ['SF', 'LF',  'GPL', 'GPS']
cond_labels = ['Small frequent', 'Large frequent',  'Grey screen L', 'Grey screen S']
experiments = ['LR', 'SR', 'LR', 'SR']
etv = etv_conds[0]
# Subject List
subjects = np.concatenate([
    ['Control_'+str(i+1) for i in range(len(all_data_epoch[0]))], 
    ['ASD_'+str(i+1) for i in range(len(all_data_epoch[1]))]
    ])
groups = np.concatenate([
    ['Control' for _ in range(len(all_data_epoch[0]))], 
    ['ASD' for _ in range(len(all_data_epoch[1]))]
    ])


df_list = []
# loop through Stimuli
for i, (etv, cond, cond_label, experiment) in enumerate(zip(etv_conds, conds, cond_labels, experiments)):
    # Cut of trials so all participants have the same amount
    min_tr = min([min([sub.shape[0] for sub in eee]) for eee in etv])
    etv = [np.stack([sub[:min_tr, ch_indices, :]*1e6 for sub in eee], axis=0) for eee in etv]
    # Average over time and re-arrange
    etv = [np.swapaxes(np.mean(group[..., pnt_range], axis=-1), 1, 2) for group in etv]
    # Prepare Data of the current stimulus    
    data = np.concatenate([np.mean(etv[0], axis=-1), np.mean(etv[1], axis=-1)], axis=0)
    
    df = pd.DataFrame(data, columns=ch_picks)
    df['Subject'] = subjects
    df['Group'] = groups
    df['Stimulus'] = [cond_label] * data.shape[0]
    df['Condition'] = [experiment] * data.shape[0]
    # Melt individual electrode columns into a single Column
    df_etv= df.melt(value_vars=ch_picks, 
                    id_vars=['Subject', 'Group', 'Stimulus', 'Condition'], 
                    value_name='ETV', var_name='Electrode')
    df_list.append(df_etv)
# Concatenate DataFrames of the different stimuli
df_etv = pd.concat(df_list)
df_etv

## Statistics

In [ ]:
dependent_variable = 'ETV'
factors = ['Group', 'Stimulus']
aov = df_etv.anova(dv=dependent_variable, between=factors, 
    detailed=True, effsize='n2')

# post-hoc test
ph_test = df_etv.pairwise_tukey(dv=dependent_variable, between='Group')
display(aov)
print('\nPost Hoc:\n')
display(ph_test)

# ETV Progression Statistics

In [ ]:
df_slope = pd.DataFrame(columns=['Subject', 'Group', 'Condition', 'Slope', 'ETVVar', 'ETVVar_detrended', 'Residual'])
groups = ['Control', 'ASD']
cond_labels = ['Small frequent', 'Large frequent', 'Grey screen L', 'Grey screen S']

for cond in range(len(etv_conds)):
    for group in range(len(etv_conds[cond])):
        for sub in range(len(etv_conds[cond][group])):
            subject = 'S' + str(sub)
            groupname = groups[group]
            conditionname = cond_labels[cond]
            etv_sample = np.mean(etv_conds[cond][group][sub][:, ch_indices, :], axis=(1, 2))
    	    # Normalize
            # etv_sample = (etv_sample - etv_sample.mean()) 
            etv_sample = (etv_sample / etv_sample.mean()) 
            
            etv_var = np.std(etv_sample)
            etv_var_detrended = np.std(detrend(etv_sample))
            # Get slope
            Y = etv_sample.reshape(-1, 1)
            X = np.arange(len(etv_sample)).reshape(-1, 1)
               
            lr = linear_model.HuberRegressor().fit(X, Y)
            line = lr.predict(X)
            slope = np.diff(np.squeeze(line))[0]
            residual = rms(etv_sample-line)

            # First half vs. second half variability
            first_half, second_half = np.array_split(etv_sample, 2)
            rel_var_red = np.std(detrend(second_half)) / np.std(detrend(first_half))

            df_slope = df_slope.append({'Subject': subject, 'Group': groupname, 
                'Condition': conditionname, 
                'Slope': slope, 'ETVVar': etv_var, 'ETVVar_detrended': etv_var_detrended, 'Residual': residual,
                'RelVarReduction': rel_var_red}, ignore_index=True)


df_slope.anova(dv='Residual', between=["Group"], detailed=True, effsize='np2')

## Statistics

In [ ]:
cols = df_slope.columns[3:]
for col in cols:
    print(col, '\n')
    display(df_slope.anova(dv=col, between=["Group", "Condition"], detailed=True, effsize='np2'))

# Prepare Data Frame with all results

In [ ]:
# ch_picks = ch_names
ch_picks = ['O1', 'Oz', 'O2']
conds = [['SF', 'LF'], ['GPL', 'GPS']]
cond_labels = ['Frequent checkers', 'Grey blanks']
scaler = 1e6
legend = ['Controls', 'ASD']

n_tr_conds = [[[calc_n_tr(epoch, conds=cond, ch_picks=ch_picks) for epoch in group] for group in all_data_epoch] for cond in conds]

subjects = np.concatenate([['Control_'+str(i+1) for i in range(len(all_data_epoch[0]))], ['ASD_'+str(i+1) for i in range(len(all_data_epoch[1]))]])
groups = np.concatenate([['Control' for _ in range(len(all_data_epoch[0]))], ['ASD' for _ in range(len(all_data_epoch[1]))]])

df = pd.DataFrame()
df['Subject'] = subjects
df['Group'] = groups

for i, cond_label in enumerate(cond_labels):
    # df[f'Raw {cond_label}'] = np.concatenate(raw_rms[i]) * scaler
    # df[f'ERP {cond_label}'] = np.concatenate(erp_rms[i]) * scaler
    df[f'Trials {cond_label}'] = np.concatenate(n_tr_conds[i])


In [ ]:

df_everything = df[['Subject', 'Group']]
# df_everything = df_everything.rename(columns={"Raw Frequent checkers": "RMS Raw", "ERP Frequent checkers": "RMS ERP"})
# AQ and EQ
df_everything['AQ'] = AQ
df_everything['EQ'] = EQ

# Add ITV
df_tmp = df_itv[df_itv['Stimulus']=='Frequent checkers']
df_tmp['Sub Idx'] = np.concatenate([np.arange(35)]*3)
df_tmp = df_tmp.pivot_table(index=['Sub Idx', 'Subject', 'Group'], columns='Electrode')
df_tmp['ROI'] = np.mean(np.stack([df_tmp[('ITV', 'O1')].values, df_tmp[('ITV', 'Oz')].values, df_tmp[('ITV', 'O2')].values], axis=0), axis=0)
df_everything['ITV_ROI'] = df_tmp['ROI'].values * 1e6

# Add ETV metrics
# frequents_bool = (df_slope['Condition'] == 'Small frequent') + (df_slope['Condition'] == 'Large frequent')
etv_slope = np.mean(np.stack([df_slope[df_slope['Condition'] == 'Small frequent']['Slope'].values, df_slope[df_slope['Condition'] == 'Large frequent']['Slope'].values], axis=0), axis=0)
etv_variability = np.mean(np.stack([df_slope[df_slope['Condition'] == 'Small frequent']['ETVVar'].values, df_slope[df_slope['Condition'] == 'Large frequent']['ETVVar'].values], axis=0), axis=0)

etv_variability_detrended = np.mean(np.stack([df_slope[df_slope['Condition'] == 'Small frequent']['ETVVar_detrended'].values, df_slope[df_slope['Condition'] == 'Large frequent']['ETVVar_detrended'].values], axis=0), axis=0)

itv_ratio = np.mean(np.stack([df_slope[df_slope['Condition'] == 'Small frequent']['RelVarReduction'].values, df_slope[df_slope['Condition'] == 'Large frequent']['RelVarReduction'].values], axis=0), axis=0)
df_everything['ETV-slope'] = etv_slope*1e6
df_everything['ETV-variability'] = etv_variability*1e6
df_everything['ETV-variability_detrended'] = etv_variability_detrended*1e6
df_everything['ITV-ratio'] = itv_ratio


df_everything.to_csv(os.path.join('../', 'processed','dataframe_asd_2.csv'))
df_everything.head()

# P1 Coefficient of Variation

In [ ]:
%matplotlib qt

ch_picks = None  # None -> Select the channel with highest P1 peak response
conds = ['SF', 'LF',]  # Select only trials in which Small Frequent and Large Frequent checkers were shown
time_range = [-0.2, 0.5]  # Default time range for this study
pnt_range = np.arange(*[np.argmin(np.abs(times-t)) for t in time_range])
scaler = 1e6  # Scale from Volts to Microvolts
baseline = (-0.1, 0)  # Baseline as described by Milne (2011)

# Calc P1 for each participant
p100_conds = [np.stack(list(map(partial(p100_milne, cond=conds, baseline=baseline, ch_name=ch_picks, verbose=0), epochs)), axis=0) 
            for epochs in all_data_epoch] 


# Convert to Pandas Series for convenience
p1_dict = dict(Controls=p100_conds[0], ASD=p100_conds[1])
ser = pd.Series(p1_dict)

## Plot

In [ ]:
fig = plt.figure(num=1)
sns.barplot(data=ser, ci=68)
plt.xticks(ticks=[0,1], labels=['Controls', 'ASD'])
plt.ylabel('Normalized Peak Amplitude Variability')
plt.xlabel('Group')
plt.title(f'P1 Variability')
plt.tight_layout()

## Statistics

In [ ]:
t,p = ttest_ind(ser[0], ser[1])
d = cohens_d(ser[0], ser[1])
print(f'P1 Variability. t: {t:.2f}, p: {p:.4f}, d: {d:.3f}')